In [1]:
from transformers import AutoModelForImageClassification
from torch.utils.data import ConcatDataset, DataLoader
import torch
import os

import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA H100 PCIe


In [3]:
base.reset_seed(42)

In [4]:
transform = base.base_transforms()
augment_transform = base.aug_transforms()

In [5]:
base.reset_seed(42)

In [6]:
model = AutoModelForImageClassification.from_pretrained(
    "aaraki/vit-base-patch16-224-in21k-finetuned-cifar10",
    num_labels=10,
)

model.to(device)
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/cifar10/teacher.pth")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [7]:
model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [8]:
dataset_part = base.get_dataset_part()

In [9]:
testing = base.unpickle(f"{os.path.expanduser('~')}/data/10/cifar-10-batches-py/test_batch")
test_data = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=False, transform=transform, device=device)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=False)

logits_test = base.generate_logits(test_dataloader, model)
testing[b"logits"] = logits_test
base.pickle_up(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/test", testing)




evaluating = base.unpickle(f"{os.path.expanduser('~')}/data/10/cifar-10-batches-py/data_batch_5")
eval_data = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=True, batch=5, transform=transform, device=device)
eval_dataloader = DataLoader(eval_data, batch_size=128, shuffle=False)

logits_eval = base.generate_logits(eval_dataloader, model)
evaluating[b"logits"] = logits_eval
base.pickle_up(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/eval", evaluating)

Generating logits for given dataset:   0%|          | 0/79 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/79 [00:00<?, ?it/s]

In [10]:
test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform)

print(base.check_acc(test, "Accuracy for test dataset is:"))
print(base.check_acc(eval, "Accuracy for eval dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy for test dataset is: 0.9508


Calculating accuracy based on the saved logits:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy for eval dataset is: 0.9583


In [11]:
base.reset_seed(42)
for index in range(1,5):
    data = base.unpickle(f"{os.path.expanduser('~')}/data/10/cifar-10-batches-py/data_batch_{index}")

    train = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=index, train=True, transform=transform, device=device)
    train_augmented = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=index, train=True, transform=augment_transform, device=device)
    
    train_dataloader = DataLoader(train, batch_size=64, shuffle=False)
    train_dataloader_augmented = DataLoader(train_augmented, batch_size=64, shuffle=False)

    logits_arr = base.generate_logits(train_dataloader, model)
    logits_arr_aug = base.generate_logits(train_dataloader_augmented, model) 

    data[b"logits"] = logits_arr
    data[b"logits_aug"] = logits_arr_aug
    base.pickle_up(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/train_batch_{index}",data)

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/157 [00:00<?, ?it/s]

In [12]:
train_aug = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)
train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform)
train_combo = ConcatDataset([train, train_aug])

In [13]:
print(base.check_acc(train, "Accuracy for train dataset is:"))
print(base.check_acc(train_aug, "Accuracy for augmeted train dataset is:"))
print(base.check_acc(train_combo, "Accuracy for combined dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/40000 [00:00<?, ?it/s]

Accuracy for train dataset is: 0.95495


Calculating accuracy based on the saved logits:   0%|          | 0/40000 [00:00<?, ?it/s]

Accuracy for augmeted train dataset is: 0.686


Calculating accuracy based on the saved logits:   0%|          | 0/80000 [00:00<?, ?it/s]

Accuracy for combined dataset is: 0.820475


In [14]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [15]:
print(len(train_aug))

28174


In [16]:
print(base.check_acc(train_aug, "Accuracy for filtered augmented dataset is:"))
print(base.check_acc(train_combo, "Accuracy for combined dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/28174 [00:00<?, ?it/s]

Accuracy for filtered augmented dataset is: 0.9615248101086108


Calculating accuracy based on the saved logits:   0%|          | 0/68174 [00:00<?, ?it/s]

Accuracy for combined dataset is: 0.9576671458327222


In [17]:
base.count_parameters(model)

model size: 327.325MB.
Total Trainable Params: 85806346.


,Modules,Parameters
0,vit.embeddings.cls_token,768
1,vit.embeddings.position_embeddings,151296
2,vit.embeddings.patch_embeddings.projection.weight,589824
3,vit.embeddings.patch_embeddings.projection.bias,768
4,vit.encoder.layer.0.attention.attention.query....,589824
...,...,...
195,vit.encoder.layer.11.layernorm_after.bias,768
196,vit.layernorm.weight,768
197,vit.layernorm.bias,768
198,classifier.weight,7680


In [18]:
train_part_cpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=1, train=True, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)

print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  204.43 ms
  1 measurement, 1000 runs , 4 threads


In [19]:
train_part_gpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=1, train=True, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)


print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  9.15 ms
  1 measurement, 1000 runs , 4 threads
